# GroupBy & Aggregate Functions
* GroupBy allows grouping of rows based on a column value.
* For example:
    * Grouping of sales data by the day of the occured sale.
    * Grouping of repeat customer data based on customer names. 
* An aggregate function follows by the GroupBy operation. 
* Aggregate functions **combine multiple rows of data into a single output** such as taking the sum of inputs or counting the number of inputs.
___

In [1]:
from pyspark.sql import SparkSession

In [2]:
# Starting a Spark session (requires some time to process)
spark = SparkSession.builder.appName("Groupbyagg").getOrCreate()

In [4]:
# Allow the first row as column name and infer Schema types
# df = spark.read.format('csv').option('header','true').load('PySparkDataSets/salesinfo.csv')
df = spark.read.csv('C:/Users/likre/Downloads/9 Python/Python DS/Datasets/salesinfo.csv', inferSchema=True, header=True)
df.printSchema()
df.show()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



___
## GroupBy

In [5]:
# Grouping based on a column
df.groupBy("Company")

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

In [6]:
# Mean of the grouped data
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [10]:
# Count of the grouped data
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [11]:
# Max of the grouped data
df.groupBy("Company").max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [12]:
# Min of the grouped data
df.groupBy("Company").min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [13]:
# Sum of the grouped data
df.groupBy("Company").sum().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



* Full documentation of other methods available on the Spark Apache reference site.

___
## Aggregate
* The .agg() method shows the aggregate for all rows in the specified dataframe column.
* Accepts arguments as a single column or create multiple aggregates via the dictionary notation.

In [7]:
# Max of a specific column
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [8]:
# Sum of a specific column
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [9]:
# Groupby + Agg
a = df.groupBy("Company")
a.agg({"Sales":'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



___
## Functions
* Function import from Spark (full documentation of other functions available on the Spark Apache reference site.)

In [10]:
# Import specific functions
from pyspark.sql.functions import countDistinct, avg, stddev, format_number

In [11]:
# Pass in a function to a column
df.select(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [12]:
# Method .alias() changes the display of the column name.
df.select(countDistinct("Sales").alias("Distinct Sales")).show()

+--------------+
|Distinct Sales|
+--------------+
|            11|
+--------------+



In [13]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [14]:
df.select(stddev("Sales")).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [15]:
# Two steps process: Column name change + Number formatting
# format_number("col_name",decimal places)
b = df.select(stddev("Sales").alias('STD'))
b.select(format_number('STD',2)).show()

+---------------------+
|format_number(STD, 2)|
+---------------------+
|               250.09|
+---------------------+



___
## OrderBy

In [16]:
# Orderby for a column (ascending by default)
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [17]:
# Orderby for a column (descending)
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



___